<a href="https://colab.research.google.com/github/team-signpin/signpin/blob/main/SignPin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SignPin

## How To Run

1. Go to your Google Drive and then click "**Shared with me**".
2. Find the "**SignPin**" folder that is shared with you.
3. **Right-click** "SignPin" and choose "**Add shortcut to drive**"
4. Cilck on "**My Drive**" from the popover men
5. Click "**Add Shortcut**" from the same menu.

Note: You only need to do this once; the first time.

In [ ]:
!pip install ipycanvas

In [ ]:
!pip install ipywidgets

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import numpy as np
from sklearn.preprocessing import normalize

In [ ]:
from ipywidgets import interact, interactive, HBox, VBox
import ipywidgets as widgets

In [ ]:
from ipycanvas import Canvas, hold_canvas

In [ ]:
with open('/content/drive/MyDrive/SignPin/data.json', 'r') as oldDataFile:
  oldData = json.loads(oldDataFile.read())

In [ ]:
canvas = Canvas(width=600, height=300)
canvas.layout.width = f'{canvas.width}px'
canvas.layout.height = f'{canvas.height}px'

In [ ]:
byNickname = {}

for signature in oldData['signatures']:
  nickname = signature['nickname']

  if nickname not in byNickname:
    byNickname[nickname] = []

  byNickname[nickname].append(signature['data'])

In [ ]:
def rotationMatrix(theta = np.pi / 2):
  cos, sin = np.cos(theta), np.sin(theta)

  return np.array(
    (
      (cos, -sin),
      (sin,  cos)
    )
  )

In [ ]:
def processBasicArray(basicArray):
  tyCount = 0
  tys = basicArray[:, 3]
  lastTy = tys[0]

  for i in range(1, len(tys)):
    ty = tys[i]

    if lastTy != ty:
      tyCount += 1
      tys[i - 1] = tyCount

    tys[i] = tyCount
    lastTy = ty

  basicArray[:, 3] = tys

  rotate90 = rotationMatrix()
  R = rotate90

  basicArray[:, 0:2] = basicArray[:, 0:2] @ R

  xCoords = basicArray[:, 0]
  yCoords = basicArray[:, 1]

  # Calculate the gradient of straightline of best fit & take x-axis gradient.
  incline = np.polyfit(xCoords, yCoords, 1)[0]

  # Calculate Radians from gradient and take negative for straightening transform.
  inclineRadians = np.arctan(incline)*1

  straightenR = rotationMatrix(inclineRadians)
  basicArray[:, 0:2] = basicArray[:, 0:2] @ straightenR

  w, h = basicArray[:, 0:2].max(axis = 0) - basicArray[:, 0:2].min(axis = 0)
  ratio = w/h

  normalizeTillColumn = 4
  basicArray[:, 0:normalizeTillColumn] = basicArray[:, 0:normalizeTillColumn] - basicArray[:, 0:normalizeTillColumn].min(axis = 0)
  basicArray[:, 0:normalizeTillColumn] = basicArray[:, 0:normalizeTillColumn] / basicArray[:, 0:normalizeTillColumn].max(axis = 0)

  return (basicArray, ratio)

In [ ]:
def renderSignature(canvas, processedData, animate = False):
  canvas.clear()
  data, ratio = processedData

  if ratio > (canvas.width / canvas.height):
    width = canvas.width
    height = canvas.width / ratio
    yOffset = (canvas.height - height) / 2
    xOffset = 0

  else:
    height = canvas.height
    width = height * ratio
    yOffset = 0
    xOffset = (canvas.width - width) / 2


  with hold_canvas(canvas):
    for i in range(len(data) - 1):
      xi, yi, tii, peni = data[i]
      xj, yj, tij, penj = data[i + 1]

      canvas.begin_path()
      canvas.stroke_style = f'hsl({peni * 270}, 100%, 50%)';

      canvas.move_to(
          xOffset + xi * width,
          yOffset + yi * height
      )

      canvas.line_to(
          xOffset + xj * width,
          yOffset + yj * height
      )

      canvas.stroke()
      canvas.close_path()

      if animate:
        canvas.sleep(
            float(speedSelector.value) * 1000 * (tij - tii)
        )

In [ ]:
def oldDataArray(nickname, index):
  basicArray = np.array([
          (row['x'], row['y'], row['ti'], row['ty'])
          for row in byNickname[nickname][index]
  ])

  return processBasicArray(basicArray)

In [ ]:
nicknameSelector = widgets.Dropdown(
    options=tuple(byNickname.keys()),
    description='Nicknames: ',
)

signatureSelector = widgets.Dropdown(
    options=tuple(range(0, len(byNickname[nicknameSelector.value]))),
    description='Sign: ',
)

speedSelector = widgets.FloatSlider(
    value=1,
    min=0.3,
    max=10.0,
    description='Speed (s): ',
)

animateButton = widgets.Button(
    description='Animate'
)

def render(animate = False):
  nickname = nicknameSelector.value
  index = int(signatureSelector.value)

  renderSignature(canvas, oldDataArray(nickname, index), animate)


def chooseNickname(nickname):
  signatureSelector.options = tuple(range(0, len(byNickname[nickname])))
  signatureSelector.value = 0

def chooseSignature(index):
  render()

def animate(_):
  render(True)


nicknameSelectorInteractive = interactive(chooseNickname, nickname = nicknameSelector)
signatureSelectorInteractive = interactive(chooseSignature, index = signatureSelector)
animateButton.on_click(animate)

VBox([
      HBox([nicknameSelectorInteractive, signatureSelectorInteractive, speedSelector, animateButton]),
      canvas
])